# 🧬 NonBDNAFinder — RAM-Adaptive Analysis & Rigorous Report Generator

**Author:** Dr. Venkata Rajesh Yella | **License:** MIT | **Version:** 2024.2

This notebook:
1. **Detects available RAM** and automatically adjusts chunk size, worker count, and execution mode.
2. **Runs all 9 Non-B DNA detectors** (G-Quadruplex, Z-DNA, Curved DNA, R-Loop, Slipped DNA, Cruciform, Triplex, i-Motif, A-philic DNA) plus hybrid and cluster detection.
3. **Generates a rigorous publication-quality report** (statistics tables, plots, per-class breakdowns, and multi-format exports) with a **single cell** execution.

---
### How to use
1. Run **Cell 1** once to install/verify dependencies.
2. Edit **Cell 2** to point to your FASTA file and set any desired options.
3. Run **Cell 3** to preview the detected system resources and the adaptive plan.
4. Run **Cell 4** (the *single analysis + report cell*) — it does everything: analysis, statistics, plots, and exports.

> **Tip:** You can also *Run All* (`Kernel → Restart & Run All`) for a fully automated pipeline.

In [1]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 1 — Dependency check & path setup
# Run once.  Safe to re-run.
# ─────────────────────────────────────────────────────────────────────────────
import sys, os, importlib

# Make sure the repository root is on PYTHONPATH so local modules resolve
_REPO_ROOT = os.path.abspath(os.path.dirname("__file__"))
if _REPO_ROOT not in sys.path:
    sys.path.insert(0, _REPO_ROOT)

_REQUIRED = [
    ("psutil",     "psutil>=5.8.0"),
    ("pandas",     "pandas>=1.3.0"),
    ("numpy",      "numpy>=1.21.0"),
    ("matplotlib", "matplotlib>=3.5.0"),
    ("seaborn",    "seaborn>=0.11.0"),
    ("openpyxl",   "openpyxl>=3.0.0"),
]

_missing = [pkg for mod, pkg in _REQUIRED if importlib.util.find_spec(mod) is None]
if _missing:
    print(f"Installing missing packages: {_missing}")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", *_missing, "-q"])

print("✅ All dependencies satisfied.")
print(f"   Python : {sys.version.split()[0]}")
import pandas as pd, numpy as np
print(f"   pandas : {pd.__version__}   numpy : {np.__version__}")

✅ All dependencies satisfied.
   Python : 3.13.5
   pandas : 2.2.3   numpy : 2.1.3


In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 2 — User configuration (Wildcard-enabled)
# ─────────────────────────────────────────────────────────────────────────────

import os
import glob
from pathlib import Path

# ── Input ────────────────────────────────────────────────────────────────────
# Accepts:
#   • Single file path
#   • Wildcards (e.g., "*.fasta", "data/*.fa")
#   • List of paths/patterns
#
# Examples:
# FASTA_INPUT = "example.fasta"
# FASTA_INPUT = "*.fasta"
# FASTA_INPUT = ["chr*.fa", "plasmid.fna"]

FASTA_INPUT = ["*.fna", "*.fasta"]   # ← change to file, wildcard, or list

def resolve_fasta_inputs(fasta_input):
    """Resolve single path, wildcard, or list into concrete file paths."""
    
    if isinstance(fasta_input, str):
        patterns = [fasta_input]
    elif isinstance(fasta_input, (list, tuple)):
        patterns = fasta_input
    else:
        raise ValueError("FASTA_INPUT must be a string or list of strings.")
    
    resolved_files = []
    
    for pattern in patterns:
        matches = glob.glob(pattern)
        if matches:
            resolved_files.extend(matches)
        elif os.path.isfile(pattern):
            resolved_files.append(pattern)
    
    # Remove duplicates and normalize paths
    resolved_files = sorted(set(Path(f).resolve() for f in resolved_files))
    
    if not resolved_files:
        raise FileNotFoundError(
            f"No FASTA files found matching: {patterns}"
        )
    
    return [str(f) for f in resolved_files]

FASTA_FILES = resolve_fasta_inputs(FASTA_INPUT)

# ── Output directory ─────────────────────────────────────────────────────────
OUTPUT_DIR = "notebook_reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ── Motif classes ─────────────────────────────────────────────────────────────
ENABLED_CLASSES = None  # None = run all 9 detectors

# ── RAM override (optional) ───────────────────────────────────────────────────
RAM_OVERRIDE_BYTES = None

# ── Export formats ───────────────────────────────────────────────────────────
EXPORT_CSV   = True
EXPORT_BED   = True
EXPORT_JSON  = True
EXPORT_EXCEL = True

# ── Report verbosity ─────────────────────────────────────────────────────────
REPORT_MODE = "full"

# ── Summary ──────────────────────────────────────────────────────────────────
print("✅ Configuration set.")
print(f"   FASTA_INPUT    : {FASTA_INPUT}")
print(f"   Files detected : {len(FASTA_FILES)}")
for f in FASTA_FILES:
    print(f"     - {f}")
print(f"   OUTPUT_DIR     : {OUTPUT_DIR}")
print(f"   ENABLED_CLASSES: {ENABLED_CLASSES or 'all 9 detectors'}")
print(f"   REPORT_MODE    : {REPORT_MODE}")

✅ Configuration set.
   FASTA_INPUT    : ['*.fna', '*.fasta']
   Files detected : 11
     - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Buchnera aphidicola.fna
     - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Candidatus Carsonella ruddii.fna
     - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Cellulomonas shaoxiangyii.fna
     - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\ecoli.fna
     - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\example.fasta
     - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\hpylori.fna
     - C:\Users\ADMIN\OneDrive - K L Unive

In [3]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 3 — System resource detection & adaptive plan preview (Wildcard-safe)
# ─────────────────────────────────────────────────────────────────────────────

import os
from Utilities.system_resource_inspector import SystemResourceInspector
from Utilities.adaptive_chunk_planner    import AdaptiveChunkPlanner

_inspector = SystemResourceInspector()

_total_ram   = _inspector.get_total_ram()
_avail_ram   = _inspector.get_available_ram()
_ram_budget  = RAM_OVERRIDE_BYTES if RAM_OVERRIDE_BYTES else _inspector.get_memory_budget()
_cpu_count   = _inspector.get_cpu_count()
_disk_free   = _inspector.get_available_disk(".")

# ── Estimate genome length from ALL FASTA files ─────────────────────────────

if not FASTA_FILES:
    raise ValueError("No FASTA files detected. Check CELL 2 configuration.")

_total_fasta_size = sum(os.path.getsize(f) for f in FASTA_FILES if os.path.exists(f))

# Rough proxy: ~1 byte per bp in plain FASTA
_est_genome = max(_total_fasta_size, 1_000)

# ── Adaptive planning ────────────────────────────────────────────────────────

_planner = AdaptiveChunkPlanner()

_plan = _planner.plan(
    genome_length = _est_genome,
    ram_budget    = _ram_budget,
    cpu_count     = _cpu_count,
)

# ── Report ───────────────────────────────────────────────────────────────────

print("━" * 58)
print("  System Resource Report")
print("━" * 58)
print(f"  Total RAM      : {_total_ram  / 1e9:.2f} GB")
print(f"  Available RAM  : {_avail_ram  / 1e9:.2f} GB")
print(f"  Memory budget  : {_ram_budget / 1e9:.2f} GB"
      + ("  [user override]" if RAM_OVERRIDE_BYTES else ""))
print(f"  CPU cores      : {_cpu_count}")
print(f"  Disk free (.)  : {_disk_free  / 1e9:.2f} GB")
print()

print("Detected FASTA files:")
for f in FASTA_FILES:
    print(f"  - {f}")

print()
print("━" * 58)
print("  Adaptive Execution Plan")
print("━" * 58)
print(f"  Estimated genome size : {_est_genome:,} bp (proxy)")
print(f"  Chunk size            : {_plan['chunk_size']:,} bp")
print(f"  Overlap               : {_plan['overlap']:,} bp")
print(f"  Workers               : {_plan['workers']}")
print(f"  Mode                  : {_plan['mode']}")
print("━" * 58)

# ── Intelligent advisory ─────────────────────────────────────────────────────

if _ram_budget < 1_000_000_000:
    print("⚠️  Low RAM detected (<1 GB budget): using conservative disk_stream mode.")
elif _ram_budget < 4_000_000_000:
    print("ℹ️  Moderate RAM (1–4 GB budget): using disk_stream with limited workers.")
else:
    print("✅ Ample RAM (≥4 GB budget): hybrid mode with parallel workers enabled.")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  System Resource Report
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Total RAM      : 17.05 GB
  Available RAM  : 5.44 GB
  Memory budget  : 3.27 GB
  CPU cores      : 4
  Disk free (.)  : 14.15 GB

Detected FASTA files:
  - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Buchnera aphidicola.fna
  - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Candidatus Carsonella ruddii.fna
  - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Cellulomonas shaoxiangyii.fna
  - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\ecoli.fna
  - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-m

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# CELL 4 — SINGLE-CELL ANALYSIS + RIGOROUS REPORT GENERATOR (BATCH SAFE)
# Supports FASTA_FILES (wildcards / multiple inputs)
# ═════════════════════════════════════════════════════════════════════════════

import os
import gc
import json
import time
import datetime
import warnings
import textwrap
import base64
from collections import defaultdict
from pathlib import Path
from IPython.display import display, HTML, Image

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

from Utilities.system_resource_inspector import SystemResourceInspector
from Utilities.adaptive_chunk_planner    import AdaptiveChunkPlanner
from Utilities.nonbscanner               import analyze_sequence as _nbf_analyze
from Utilities.utilities                 import (
    read_fasta_file, export_to_csv, export_to_bed,
    export_to_json, export_to_excel,
)

# ─────────────────────────────────────────────────────────────────────────────
# 0. Validate FASTA inputs
# ─────────────────────────────────────────────────────────────────────────────

if not FASTA_FILES:
    raise ValueError("No FASTA files detected. Check CELL 2.")

print("📂 FASTA input(s):")
for f in FASTA_FILES:
    print("   -", f)

# ─────────────────────────────────────────────────────────────────────────────
# 1. Prepare output directory
# ─────────────────────────────────────────────────────────────────────────────

_OUT = Path(OUTPUT_DIR)
_OUT.mkdir(parents=True, exist_ok=True)

_RUN_TS = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
_BASE = _OUT / _RUN_TS
_BASE.mkdir(parents=True, exist_ok=True)

print(f"\n📂 Output directory : {_BASE}")

# ─────────────────────────────────────────────────────────────────────────────
# 2. Adaptive resource planning (batch aware)
# ─────────────────────────────────────────────────────────────────────────────

_total_input_size = sum(
    os.path.getsize(f) for f in FASTA_FILES if os.path.exists(f)
)
_est_genome = max(_total_input_size, 1_000)

_insp   = SystemResourceInspector()
_budget = RAM_OVERRIDE_BYTES if RAM_OVERRIDE_BYTES else _insp.get_memory_budget()
_cpus   = _insp.get_cpu_count()

_plan = AdaptiveChunkPlanner().plan(
    genome_length=_est_genome,
    ram_budget=_budget,
    cpu_count=_cpus,
)

CHUNK_SIZE    = _plan["chunk_size"]
CHUNK_OVERLAP = _plan["overlap"]
N_WORKERS     = _plan["workers"]
EXEC_MODE     = _plan["mode"]

print(f"⚙️  Adaptive plan: chunk={CHUNK_SIZE:,}, overlap={CHUNK_OVERLAP:,}, "
      f"workers={N_WORKERS}, mode={EXEC_MODE}")

# ─────────────────────────────────────────────────────────────────────────────
# 3. Parse all FASTA files
# ─────────────────────────────────────────────────────────────────────────────

_sequences = {}

for _fasta in FASTA_FILES:
    if not os.path.exists(_fasta):
        raise FileNotFoundError(f"Missing FASTA: {_fasta}")

    _seqs = read_fasta_file(_fasta)
    if not _seqs:
        print(f"⚠️ No sequences parsed from {_fasta}")
        continue

    _prefix = Path(_fasta).stem
    for _name, _seq in _seqs.items():
        _sequences[f"{_prefix}|{_name}"] = _seq

if not _sequences:
    raise ValueError("No sequences loaded from input FASTA files.")

print(f"\n🔬 Total sequences loaded: {len(_sequences)}")

# ─────────────────────────────────────────────────────────────────────────────
# 4. Run motif detection
# ─────────────────────────────────────────────────────────────────────────────

_all_motifs = []
_seq_stats  = {}
_t0 = time.perf_counter()

for idx, (seq_name, seq) in enumerate(_sequences.items(), 1):
    print(f"[{idx}/{len(_sequences)}] {seq_name[:60]} "
          f"({len(seq):,} bp)", end="")

    _t_seq = time.perf_counter()

    motifs = _nbf_analyze(
        sequence=seq,
        sequence_name=seq_name,
        use_chunking=True,
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        use_parallel_chunks=(EXEC_MODE == "hybrid"),
        enabled_classes=ENABLED_CLASSES,
    )

    elapsed = time.perf_counter() - _t_seq
    print(f" → {len(motifs):,} motifs ({elapsed:.1f}s)")

    _all_motifs.extend(motifs)

    _seq_stats[seq_name] = {
        "length": len(seq),
        "motifs": len(motifs),
        "elapsed_s": elapsed,
    }

    gc.collect()

_total_elapsed = time.perf_counter() - _t0
print(f"\n✅ Total motifs: {len(_all_motifs):,} "
      f"in {_total_elapsed:.1f} seconds")

# ─────────────────────────────────────────────────────────────────────────────
# 5. Build DataFrame
# ─────────────────────────────────────────────────────────────────────────────

_df = pd.DataFrame(_all_motifs) if _all_motifs else pd.DataFrame()

for col, default in [
    ("Class", "Unknown"),
    ("Subclass", "Other"),
    ("Start", 0),
    ("End", 0),
    ("Length", 0),
    ("Score", 0.0),
    ("Strand", "+"),
    ("Sequence_Name", ""),
]:
    if col not in _df.columns:
        _df[col] = default

if not _df.empty and "Length" in _df.columns:
    mask = _df["Length"] == 0
    _df.loc[mask, "Length"] = (
        _df.loc[mask, "End"] - _df.loc[mask, "Start"]
    ).clip(lower=0)

# ─────────────────────────────────────────────────────────────────────────────
# 6. Summary statistics
# ─────────────────────────────────────────────────────────────────────────────

_total_bp = sum(len(s) for s in _sequences.values())
_total_motifs = len(_df)
_class_counts = _df["Class"].value_counts().to_dict() if not _df.empty else {}

_density = (_total_motifs / (_total_bp / 1000)) if _total_bp else 0

print("\n════════════════════════════════════════════════════")
print("ANALYSIS SUMMARY")
print("════════════════════════════════════════════════════")
print("Input files :", ", ".join(Path(f).name for f in FASTA_FILES))
print("Sequences   :", len(_sequences))
print("Total bp    :", f"{_total_bp:,}")
print("Motifs      :", f"{_total_motifs:,}")
print("Density     :", f"{_density:.3f} motifs/kb")
print("════════════════════════════════════════════════════")

# ─────────────────────────────────────────────────────────────────────────────
# 7. Simple class distribution plot
# ─────────────────────────────────────────────────────────────────────────────

sns.set_theme(style="whitegrid")
_fig_paths = []

if _class_counts:
    cls_df = pd.DataFrame(
        sorted(_class_counts.items(), key=lambda x: x[1], reverse=True),
        columns=["Class", "Count"]
    )

    fig, ax = plt.subplots(figsize=(9, 4))
    ax.barh(cls_df["Class"][::-1], cls_df["Count"][::-1])
    ax.set_xlabel("Motif Count")
    ax.set_title("Motif Class Distribution")

    plt.tight_layout()
    fp = str(_BASE / "class_distribution.png")
    fig.savefig(fp, dpi=150)
    plt.close(fig)

    _fig_paths.append(("Class Distribution", fp))
    display(Image(fp))

# ─────────────────────────────────────────────────────────────────────────────
# 8. Export files
# ─────────────────────────────────────────────────────────────────────────────

_export_paths = {}

if not _df.empty:
    motif_list = _df.to_dict(orient="records")

    if EXPORT_CSV:
        p = str(_BASE / "motifs.csv")
        export_to_csv(motif_list, p)
        _export_paths["CSV"] = p

    if EXPORT_BED:
        p = str(_BASE / "motifs.bed")
        export_to_bed(motif_list, p)
        _export_paths["BED"] = p

    if EXPORT_JSON:
        p = str(_BASE / "motifs.json")
        export_to_json(motif_list, p)
        _export_paths["JSON"] = p

    if EXPORT_EXCEL:
        p = str(_BASE / "motifs.xlsx")
        export_to_excel(motif_list, p)
        _export_paths["Excel"] = p

# ─────────────────────────────────────────────────────────────────────────────
# 9. Generate HTML report
# ─────────────────────────────────────────────────────────────────────────────

def _img_tag(path):
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode()
    return f'<img src="data:image/png;base64,{b64}" style="max-width:100%"/>'

_fig_html = "".join(
    f"<h3>{title}</h3>{_img_tag(path)}"
    for title, path in _fig_paths
)

_html = f"""
<html>
<head><title>NonBDNAFinder Report</title></head>
<body style="font-family:Arial;max-width:1000px;margin:auto;">
<h1>Non-B DNA Motif Report</h1>
<p><b>Input files:</b> {", ".join(Path(f).name for f in FASTA_FILES)}</p>
<p><b>Total motifs:</b> {_total_motifs:,}</p>
<p><b>Total bp:</b> {_total_bp:,}</p>
<p><b>Density:</b> {_density:.3f} motifs/kb</p>
<h2>Figures</h2>
{_fig_html}
</body>
</html>
"""

_report_path = str(_BASE / "report.html")
with open(_report_path, "w", encoding="utf-8") as f:
    f.write(_html)

print(f"\n📄 HTML report written to: {_report_path}")
display(HTML(f'<a href="{_report_path}" target="_blank">Open Report</a>'))

📂 FASTA input(s):
   - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Buchnera aphidicola.fna
   - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Candidatus Carsonella ruddii.fna
   - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\Cellulomonas shaoxiangyii.fna
   - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\ecoli.fna
   - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\example.fasta
   - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-main\hpylori.fna
   - C:\Users\ADMIN\OneDrive - K L University\Desktop\CURRENT_RESEARCH\NonBDNATool\FINALIZATION\Genomes\NonBDNAFinder-mai